In [1]:
import json
import os
import random
import time

In [2]:
import h3
import geopandas as gpd
import pandas as pd

In [3]:
from kroger_api import KrogerAPI
from kroger_api.utils.env import load_and_validate_env, get_zip_code
from shapely.geometry import Point
from tqdm.notebook import tqdm

## Get State Data

In [4]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [5]:
states_df = states_df[["STUSPS", "NAME", "geometry"]]

## Get Population data

In [6]:
state_populations = pd.read_excel(
    "data/NST-EST2024-POP.xlsx", sheet_name=None, engine="openpyxl"
)

ERROR! Session/line number was not unique in database. History logging moved to new session 253


In [7]:
state_populations_df = state_populations["NST-EST2024-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [8]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

## Get Store Data

In [ ]:
load_and_validate_env(["KROGER_CLIENT_ID", "KROGER_CLIENT_SECRET"])
zip_code = get_zip_code(default="10001")

In [10]:
kroger = KrogerAPI()
token_info = kroger.authorization.get_token_with_client_credentials("product.compact")

Found saved token, will test if it's still valid...
Token appears invalid, requesting a new one


In [11]:
res = 3
distance = 0.5

usa_geom = states_df[~states_df["STUSPS"].isin(["PR", "VI", "GU", "MP", "AS"])][
    "geometry"
].union_all()

h3_shape = h3.geo_to_h3shape(usa_geom.buffer(distance=distance))
h3_cells = h3.h3shape_to_cells(h3_shape, res=res)
geojson = h3.cells_to_geo(h3_cells)

In [12]:
stores = []
for cell in tqdm(h3_cells, desc="Parsing Stores"):
    lat, lng = h3.cell_to_latlng(cell)
    locations = kroger.location.search_locations(
        lat=lat, lon=lng, radius_in_miles=100, limit=200
    )
    if len(locations["data"]):
        for location in locations["data"]:
            location["geometry"] = Point(
                location["geolocation"]["longitude"],
                location["geolocation"]["latitude"],
            )
            location["STUSPS"] = location["address"]["state"]
            stores.append(location)
    else:
        continue
    time.sleep(random.uniform(0.01, 0.5))

Parsing Stores:   0%|          | 0/911 [00:00<?, ?it/s]

In [45]:
stores_gdf = gpd.GeoDataFrame(stores, crs=4326)
stores_gdf = stores_gdf.drop_duplicates("locationId")
stores_gdf.to_file("data/all_stores.gpkg")

In [25]:
stores_gdf = gpd.read_file("data/all_stores.gpkg")

In [26]:
len(stores_gdf)

2866

In [27]:
stores_gdf = stores_gdf[
    ~stores_gdf[["phone", "departments"]].isnull().all(axis=1)
].reset_index(drop=True)

In [28]:
len(stores_gdf)

2832

In [49]:
most_common_state_df = (
    stores_gdf.groupby("STUSPS")["chain"]
    .value_counts()
    .groupby(level=0)
    .idxmax()
    .apply(lambda x: x[1])
    .reset_index(name="most_common_chain")
)

In [53]:
most_common_state_df.groupby("most_common_chain").size().reset_index(
    name="COUNT"
).sort_values("COUNT", ascending=False)

,most_common_chain,COUNT
7,KROGER,15
5,HART,5
11,SMITHS,5
2,FRED,4
3,FRYS,1
1,DILLONS,1
0,BAKERS,1
4,GERBES,1
6,KINGSOOPERS,1
8,MARIANOS,1


## Combine With States

In [56]:
most_common_state_gdf = states_with_population_df.merge(
    most_common_state_df, on="STUSPS", how="left"
)

In [57]:
most_common_state_gdf = most_common_state_gdf.to_crs(9311)
most_common_state_gdf.to_file(f"data/Most_Common_Kroger_Brand_Per_State.gpkg")